# FUNCIONES

In [1]:
import os
import numpy as np
import pprint
import copy
from math import sqrt
from scipy.linalg import solve_triangular

### Creación matrices

In [2]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

### Factorización QR

In [3]:
def house(x):
    """
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
    """
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

In [4]:
def factorizacion_QR(A):
    """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: A_r_v  Matriz (mxn) con la información escencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
    """
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [5]:
def QT_C(A_r_v,C):
    """
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: M       Matriz con numero al azar
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [6]:
def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

### Funciones para solución de Sistemas de Ecuaciones Lineales

In [7]:
def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    A_r_v=factorizacion_QR(A)
    m=A_r_v.shape[0]
    #Q=np.transpose(QT_C(A_r_v,np.eye(m)))
    #R=np.transpose(Q)@A
    n=A_r_v.shape[0]
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(m):
        Qj=Q_j(A_r_v,j)
        Q=Q@Qj
        R=Q_j(A_r_v,j)@R
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x

#### Eliminación por bloques

In [8]:
def bloques(A, b=False, n1=False, n2=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')

    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [9]:
def eliminacion_bloques(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        ys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

# Prueba Unitaria

## Eliminación por bloques con QR considerando sistemas sin solución

### Ejemplo 1 - Matriz 3 x 3 (Fila con ceros)

Empezaremos por generar un sistema de ecuaciones lineales sin solución.

In [10]:
# Generamos una matriz 3 x 3
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[-2], [0], [5]], dtype='d')
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

A:
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])
b:
array([[-2.],
       [ 0.],
       [ 5.]])


**Numpy**

Utilizaremos la función de numpy *np.linalg.solve(A,b)*  para validar que el sistema de ecuaciones efectivamente no tiene solución.

In [11]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

Podemos observar que la función de numpy nos arroja un error al tratar de resolver un sistema de ecuaciones lineales sin solución.

El error se refiere a que la matriz A es una matriz singular.

**Implementación Programadores - Eliminación por bloques con QR**

Utilizaremos la función eliminacion_bloques implementada por los programadores para validar sus funcionalidad cuando trata de resolver un sistema de ecuaciones lineales sin solución.

In [12]:
eliminacion_bloques(A,b)

NameError: name 'sys' is not defined

Podemos observar que la función nos arroja un error al tratar de resolver un sistema de ecuaciones lineales sin solución al igual que numpy. Sin embargo, no existe ningun mensaje que le de visibilidad al usuario del tipo de error al que se esta enfrentando.

### Ejemplo 2 - Matriz 10^2 x 10^2 (Fila con ceros)

Empezamos por generar un sistema de ecuaciones lineales de 10^2 x 10^2

In [13]:
m = 100
n = 100
A = np.round(crea_matriz(m, n, 10, -10,False), 2)
b = np.round(crea_matriz(m, 1, 10, -10,False), 2)
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

A:
array([[-9.63, -6.12, -8.03, ..., -4.43,  6.24, -1.59],
       [-0.26,  7.23,  8.82, ..., -0.24,  2.05, -4.42],
       [ 3.99,  6.74,  4.2 , ...,  2.01, -6.11,  1.66],
       ...,
       [ 5.8 ,  0.46, -0.6 , ...,  7.06,  2.02,  5.08],
       [ 8.47, -2.51, -8.59, ...,  0.13, -7.54, -3.79],
       [-2.25, -8.52,  9.51, ...,  0.42,  4.89,  1.4 ]])
b:
array([[ 9.95],
       [ 6.96],
       [-9.03],
       [ 1.18],
       [ 6.34],
       [-4.46],
       [-1.12],
       [-9.42],
       [ 8.98],
       [-2.25],
       [ 9.35],
       [-6.25],
       [ 8.36],
       [-7.46],
       [-6.79],
       [ 1.49],
       [-1.86],
       [-5.59],
       [ 3.2 ],
       [-9.51],
       [-2.16],
       [ 9.24],
       [ 2.69],
       [-0.26],
       [-5.55],
       [-6.02],
       [ 4.71],
       [-1.41],
       [ 3.1 ],
       [ 9.04],
       [ 6.73],
       [-0.1 ],
       [-3.03],
       [ 9.88],
       [ 3.36],
       [ 4.11],
       [-3.69],
       [ 5.04],
       [ 7.57],
       [ 1.8 ],
     

Actualizaremos la ultima fila de la matriz con puros ceros para volverlo un sistema de ecuaciones lineales sin solución.

In [14]:
A[-1] = np.zeros(n)

In [15]:
print("A:")
pprint.pprint(A)

A:
array([[-9.63, -6.12, -8.03, ..., -4.43,  6.24, -1.59],
       [-0.26,  7.23,  8.82, ..., -0.24,  2.05, -4.42],
       [ 3.99,  6.74,  4.2 , ...,  2.01, -6.11,  1.66],
       ...,
       [ 5.8 ,  0.46, -0.6 , ...,  7.06,  2.02,  5.08],
       [ 8.47, -2.51, -8.59, ...,  0.13, -7.54, -3.79],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])


**Numpy**

Utilizaremos la función de numpy *np.linalg.solve(A,b)*  una vez mas para validar que el sistema de ecuaciones dado efectivamente no tiene solución.

In [16]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

Podemos observar que la función de numpy nos arroja el mismo error que en el Ejemplo 1.

**Implementación Programadores - Eliminación por bloques con QR**

Utilizaremos la función eliminacion_bloques implementada por los programadores para validar sus funcionalidad cuando trata de resolver un sistema de ecuaciones lineales sin solución.

In [17]:
eliminacion_bloques(A,b)

NameError: name 'sys' is not defined

Una vez mas, podemos observar que la función nos arroja un error al tratar de resolver un sistema de ecuaciones lineales sin solución al igual que numpy. Sin embargo, no existe ningun mensaje que le de visibilidad al usuario del tipo de error al que se esta enfrentando.

### Ejemplo 3 - Matriz 2 x 2 (Sistema incompatible - Rectas Paralelas)

Empezaremos por generar un sistema de ecuaciones lineales sin solución (Rectas Paralelas).

In [18]:
# Generamos una matriz 2 x 2 incompatible
A = np.array([[11, 4], [132, 48]], dtype='d')
b = np.array([[7], [-1]], dtype='d')
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

A:
array([[ 11.,   4.],
       [132.,  48.]])
b:
array([[ 7.],
       [-1.]])


**Numpy**

Utilizaremos la función de numpy *np.linalg.solve(A,b)*  una vez mas para validar que el sistema de ecuaciones dado efectivamente no tiene solución.

In [19]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

Podemos observar que la función de numpy nos arroja el mismo error que en el Ejemplo 1 y 2.

**Implementación Programadores - Eliminación por bloques con QR**

Utilizaremos la función eliminacion_bloques implementada por los programadores para validar sus funcionalidad cuando trata de resolver un sistema de ecuaciones lineales sin solución.

In [20]:
eliminacion_bloques(A,b)

NameError: name 'sys' is not defined

Una vez mas, podemos observar que la función nos arroja un error al tratar de resolver un sistema de ecuaciones lineales sin solución al igual que numpy. Sin embargo, no existe ningun mensaje que le de visibilidad al usuario del tipo de error al que se esta enfrentando.

### Ejemplo 4 - Matriz 4 x 3 no cuadrada

In [21]:
m = 4
n = 3
A = np.round(crea_matriz(m, n, 10, -10,False), 2)
b = np.round(crea_matriz(m, 1, 10, -10,False), 2)
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

A:
array([[-1.67, -9.95, -4.9 ],
       [-8.67, -2.01,  1.69],
       [ 1.85,  6.62,  9.91],
       [-5.53, -9.39, -8.54]])
b:
array([[-8.54],
       [-4.47],
       [ 8.08],
       [ 8.66]])


**Numpy**

Utilizaremos la función de numpy *np.linalg.solve(A,b)* para observar que hace cuando se enfrenta a una matriz no cuadrada.

In [22]:
np.linalg.solve(A,b)

LinAlgError: Last 2 dimensions of the array must be square

La función nos arroja un error en el que nos dice que las dimensiones de la matriz deben de ser cuadradas.

**Implementación Programadores - Eliminación por bloques con QR**

Utilizaremos la función eliminacion_bloques implementada por los programadores para observar que hace cuando se enfrenta a una matriz que no es cuadrada.

In [23]:
eliminacion_bloques(A,b)

LinAlgError: Last 2 dimensions of the array must be square

Para este ejemplo podemos apreciar que la función nos arroja el mismo error que numpy. Sería bueno agregar una validación previa a la ejecución, en la que garanticemos que el input es una matriz cuadrada.

## Resumen de Hallazgos

La función eliminacion_bloques(A,b) debería de ser capaz de identificar cuando se enfrenta a un sistema de ecuaciones lineales para los cuales no existe solución. En estos casos la función debería de dar como output un mensaje "Sistema de ecuaciones sin solución" o "Matriz singular" por ejemplo. 

Por otra parte, tambien debería validar las dimensiones de la matriz de entrada, si las dimensiones de la matriz de entrada no son cuadradas deberíamos de recibir como output un mensaje "Las dimensiones de la matriz (A) no son cuadradas, por favor revisa" por dar algun ejemplo.

La finalidad de agregar estas funcionalidades a la función es poder dar retroalimentación al usuario respecto a la naturaleza de los errores que se pudieran presentar al utilizar la función.

Dados estos hallazgos levantaremos un issue con las siguientes acciones:

**TO DO.** 

+ Agregar validación para sistema de ecuaciones sin solución. 

+ Agregar validación para matrices no cuadradas.
